# PGM Compilers Advanced Topics

Recall a PGM compiler is a function that takes a PGM and compiles it into an arithmetic circuit, specifically a `PGMCircuit` object.

There are several PGM compilers provided by CK, and it is possible to write custom PGM compilers. A PGM compiler is a callable with the signature:

`def y_pgm_compiler(pgm: PGM, *, const_parameters: bool = True) -> PGMCircuit:`

That is, the callable takes one argument, a PGM to compile, and an optional keyword argument `const_parameters` which should default to True.

If `const_parameters` is True, then the potential function parameters will be circuit constants, otherwise they will be circuit variables.

If the potential function parameters are represented as constants, then there are various optimisations that are available to subsequent processes. If the potential function parameters are represented as circuit variables, then potential functions can be changed dynamically without needing to recompile.


CK provides many PGM compilers using different algorithms: Variable Elimination, Factor Elimination, and Recursive Conditioning. These algorithms can use different heuristics for the process order of random variables, thus there are many different PGM compilers provided.

CK also provides a way to use the ACE Bayesian network compiler as a PGM compiler. To use ACE you must have a working copy of ACE (see http://reasoning.cs.ucla.edu/ace/).

You can enable CK to access ACE either by setting the OS environment variable `CK_ACE_LOCATION` to configure the path of
the default ACE installation directory, or you can copy ACE to the default ACE installation directory using the method `ck.pgm_compiler.ace.copy_ace_to_default_location`.

See the module `ck.utils.local_config` to understand more about configuration variable management.

Standard PGM compilers are members of the `NamedPGMCompiler` enum.

Here are the available named PGM compilers:

In [1]:
from ck.pgm_compiler import NamedPGMCompiler

for compiler in NamedPGMCompiler:
    print(compiler.name)

VE_MIN_DEGREE
VE_MIN_DEGREE_THEN_FILL
VE_MIN_FILL
VE_MIN_FILL_THEN_DEGREE
VE_MIN_WEIGHTED_DEGREE
VE_MIN_WEIGHTED_FILL
VE_MIN_TRADITIONAL_WEIGHTED_FILL
FE_MIN_DEGREE
FE_MIN_DEGREE_THEN_FILL
FE_MIN_FILL
FE_MIN_FILL_THEN_DEGREE
FE_MIN_WEIGHTED_DEGREE
FE_MIN_WEIGHTED_FILL
FE_MIN_TRADITIONAL_WEIGHTED_FILL
FE_BEST_JOINTREE
RC_MIN_DEGREE
RC_MIN_DEGREE_THEN_FILL
RC_MIN_FILL
RC_MIN_FILL_THEN_DEGREE
RC_MIN_WEIGHTED_DEGREE
RC_MIN_WEIGHTED_FILL
RC_MIN_TRADITIONAL_WEIGHTED_FILL
ACE


Named compilers with names that start with "VE" use variable elimination, that start with "FE" use factor elimination, and that start with "RC" use recursive conditioning. The ordering heuristics are:

| Order Heuristic Name          | Explanation                                        |
|-------------------------------|----------------------------------------------------|
| MIN_DEGREE                    | "minimum degree", ties broken arbitrarily          |
| MIN_DEGREE_THEN_FILL          | "minimum degree", ties broken using "minimum fill" |
| MIN_FILL                      | "minimum fill", ties broken arbitrarily            |
| MIN_FILL_THEN_DEGREE          | "minimum fill", ties broken using "minimum degree" |
| MIN_WEIGHTED_DEGREE           | "minimum weighted degree"                          |
| MIN_WEIGHTED_FILL             | a CK custom version of "minimum weighted fill"     |
| MIN_TRADITIONAL_WEIGHTED_FILL | the traditional version of "minimum weighted fill" |


Many text books are available that explain the VE, FE and RC algorithms, as well as the main ordering heuristics. For example, see:
["Modeling and Reasoning with Bayesian Networks Book", Adnan Darwiche (2009)](
https://www.cambridge.org/core/books/modeling-and-reasoning-with-bayesian-networks/8A3769B81540EA93B525C4C2700C9DE6).

Compiler "FE_BEST_JOINTREE" tries factor elimination with multiple ordering heuristics and uses the result where the join tree has the smallest maximum cluster size.

Compiler "ACE" is the ACE compiler, but needs ACE installed and copied to CK (see http://reasoning.cs.ucla.edu/ace/ and `ck.pgm_compiler.ace.copy_ace_to_default_location`).

The default PGM compiler is available as `DEFAULT_PGM_COMPILER`, which is a `NamedPGMCompiler` enum member.

In [2]:
from ck.pgm_compiler import DEFAULT_PGM_COMPILER

DEFAULT_PGM_COMPILER.name

'FE_BEST_JOINTREE'

A named compiler is directly callable as a PGM compiler:

In [3]:
from ck.example import Alarm

pgm = Alarm()
pgm_cct = DEFAULT_PGM_COMPILER(pgm)

Note the different compilers will have different performance characteristics. The following code shows the number of arithmetic operations for each circuit resulting from each named compiler (from the given PGM).


In [4]:
for compiler in NamedPGMCompiler:
    try:
        pgm_cct = compiler(pgm)
    except Exception as err:
        print(compiler.name, 'FAILED:', err)
        continue

    cct_size = pgm_cct.circuit_top.circuit.number_of_operations
    print(f'{compiler.name:>32} {cct_size}')

                   VE_MIN_DEGREE 2554
         VE_MIN_DEGREE_THEN_FILL 2554
                     VE_MIN_FILL 2471
         VE_MIN_FILL_THEN_DEGREE 2471
          VE_MIN_WEIGHTED_DEGREE 2268
            VE_MIN_WEIGHTED_FILL 2423
VE_MIN_TRADITIONAL_WEIGHTED_FILL 2423
                   FE_MIN_DEGREE 2447
         FE_MIN_DEGREE_THEN_FILL 2447
                     FE_MIN_FILL 2459
         FE_MIN_FILL_THEN_DEGREE 2459
          FE_MIN_WEIGHTED_DEGREE 2304
            FE_MIN_WEIGHTED_FILL 2330
FE_MIN_TRADITIONAL_WEIGHTED_FILL 2330
                FE_BEST_JOINTREE 2304
                   RC_MIN_DEGREE 3882
         RC_MIN_DEGREE_THEN_FILL 3882
                     RC_MIN_FILL 3883
         RC_MIN_FILL_THEN_DEGREE 3883
          RC_MIN_WEIGHTED_DEGREE 3260
            RC_MIN_WEIGHTED_FILL 3763


RC_MIN_TRADITIONAL_WEIGHTED_FILL 3763


                             ACE 1433
